# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

    ● сalls — количество звонков,
    ● minutes — суммарная длительность звонков в минутах,
    ● messages — количество sms-сообщений,
    ● mb_used — израсходованный интернет-трафик в Мб,
    ● is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбейте данные на выборки

In [4]:
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']

features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.25, random_state=123456)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, 
                                                                              test_size=0.25, random_state=123456)

## Исследуйте модели
### Модель "решающее дерево"

In [5]:
tree_model = None
best_result = 0
for depth in range(1, 9):
    model = DecisionTreeClassifier(max_depth=depth, random_state=123456)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        tree_model = model
        best_tree_result = result

print("Accuracy наилучшей модели на валидационной выборке:", best_tree_result)

Accuracy наилучшей модели на валидационной выборке: 0.7827529021558872


### Модель "случайный лес"

In [6]:
forest_model = None
best_result = 0
for est in range(1, 50):
    model = RandomForestClassifier(random_state=123456, n_estimators=est)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        forest_model = model
        best_forest_result = result

print("Accuracy наилучшей модели на валидационной выборке:", best_forest_result)

Accuracy наилучшей модели на валидационной выборке: 0.8076285240464345


### Модель "логическая регрессия"

In [7]:
logist_model = None
best_result = 0
for more_c in range(1,50):
    model = LogisticRegression(solver='lbfgs', random_state=123456, C=more_c, max_iter=1000)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        logist_model = model
        best_logist_result = result

print("Accuracy наилучшей модели на валидационной выборке:", best_logist_result)

Accuracy наилучшей модели на валидационной выборке: 0.7728026533996684


### Вывод

В каждой модели мы подобрали самый подходящий гиперпараметр

На валидационных выборках лучше всех себя показала модель случайного леса с аccuracy = 0.8, посмотрим как она себя покажет на тестовой выборке, модель решающего дерева не далеко ушла от модели случайного леса, а её догоняет модель логической регрессии

## Проверьте модели на тестовой выборке

### Проверка решающего дерева

In [8]:
predictions_train = tree_model.predict(features_train)
train = accuracy_score(target_train, predictions_train)
predictions_test = tree_model.predict(features_test) 
test = accuracy_score(target_test, predictions_test)

print("Accuracy")
print("Обучающая выборка:", train) 
print("Тестовая выборка:", test)

Accuracy
Обучающая выборка: 0.8572219147758716
Тестовая выборка: 0.7810945273631841


### Проверка случайного леса
Похоже случайный лес очень усердно зубрил учебную выборку

In [9]:
predictions_train = forest_model.predict(features_train)
train = accuracy_score(target_train, predictions_train)
predictions_test = forest_model.predict(features_test) 
test = accuracy_score(target_test, predictions_test)

print("Accuracy")
print("Обучающая выборка:", train) 
print("Тестовая выборка:", test)

Accuracy
Обучающая выборка: 0.9994465965688988
Тестовая выборка: 0.8109452736318408


### Проверка логической регрессии

In [10]:
predictions_train = logist_model.predict(features_train)
train = accuracy_score(target_train, predictions_train)
predictions_test = logist_model.predict(features_test) 
test = accuracy_score(target_test, predictions_test)

print("Accuracy")
print("Обучающая выборка:", train) 
print("Тестовая выборка:", test)

Accuracy
Обучающая выборка: 0.7365799667957942
Тестовая выборка: 0.763681592039801


Лучше всех себя показала модель случайного леса (что на тестовых хороша, что на валидационных), а вот модель логической регрессии не справляется